In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

# login to mt5

In [2]:
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

# display data on MetaTrader 5 version
print(mt5.version())
# connect to the trade account without specifying a password and a server
# account=51859831
# password="Ali@1234"

#account = 51863584
#password = "Test@10000"
#server="Alpari-MT5-Demo"

account = 62168
password = "q0M26Jg4A4"
server="FusionMarkets-Demo"
symbol = 'EURUSD'

authorized=mt5.login(account, server=server,password=password)  # the terminal database password is applied if connection data is set to be remembered
if authorized:
    print("connected to account #{}".format(account))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

# now connect to another trading account specifying the password
authorized=mt5.login(account, server=server,password=password)
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))


MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.45
(500, 4040, '20 Oct 2023')
connected to account #62168
AccountInfo(login=62168, trade_mode=0, leverage=500, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=14363.53, credit=0.0, profit=0.0, equity=14363.53, margin=0.0, margin_free=14363.53, margin_level=0.0, margin_so_call=90.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DEMO_002', server='FusionMarkets-Demo', currency='USD', company='Fusion Markets Pty Ltd')
Show account_info()._asdict():
  login=62168
  trade_mode=0
  leverage=500
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=14363.53
  credit=0.0
  profit=0.0
  equity=14363.53
  margin=0.0
  margin_free=14363.53
  margin_level=0.0
  margin_so_

# define limit order

In [3]:
def buy(symbol, lot):
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 2
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
#        "sl": sl,
#        "tp": tp,
        "deviation": deviation,
        "magic": 234000,
        "comment": "python script",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    # send a trading request
    result = mt5.order_send(request)
    return result

In [4]:
def sell(symbol , lot):
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 5
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
#        "sl": sl,
#        "tp": tp,
        "deviation": deviation,
        "magic": 234000,
        "comment": "python script",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    # send a trading request
    result = mt5.order_send(request)
    return result

In [5]:
def close_all_position(symbol, open_positions):
    point = mt5.symbol_info(symbol).point
    for position in open_positions:
        direction = position.request.type
        lot = position.request.volume
        position_id = position.order

        if direction==1 :
            price=mt5.symbol_info_tick(symbol).ask
            deviation=5
            request={
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": mt5.ORDER_TYPE_BUY,
                "position": position_id,
                "price": price,
                "deviation": deviation,
                "magic": 234000,
                "comment": "python script close",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            mt5.order_send(request)

        elif direction==0:
            price=mt5.symbol_info_tick(symbol).bid
            deviation=2
            request={
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": mt5.ORDER_TYPE_SELL,
                "position": position_id,
                "price": price,
                "deviation": deviation,
                "magic": 234000,
                "comment": "python script close",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            mt5.order_send(request)

# without maximum step

In [6]:
from time import sleep

def run(first_order, lot, a, x, symbol, spread):
    point = mt5.symbol_info(symbol).point
    positions = []

    if (first_order == buy) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread):
        entry = mt5.symbol_info_tick(symbol).ask
        top = entry + a * x *point*10
        bottom = entry - (a+1) * x *point*10
        position = buy(symbol,lot=lot)
        positions.append(position)
        step = 1
        print('order 1 executed ')
        sleep(1)
        while first_order==buy:
            if ((mt5.positions_get()[-1]._asdict()['type'] == 0) and (mt5.symbol_info_tick(symbol).bid <= entry - x*point*10) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                lot = 2 * lot
                position = sell(symbol, lot)
                positions.append(position)
                step = step + 1
                print('order %i executed'%(step))
                sleep(0.5)
            elif ((mt5.positions_get()[-1]._asdict()['type'] == 1) and (mt5.symbol_info_tick(symbol).ask >= entry) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                lot = 2 * lot
                position = buy(symbol, lot)
                positions.append(position)
                sleep(0.5)
                step = step + 1
                print('order %i executed'%(step))
            if ((mt5.symbol_info_tick(symbol).ask < bottom) or (mt5.symbol_info_tick(symbol).bid >= top)):
                close_all_position(symbol ,positions)
                print('all %i orders closed'%(step))
                break


    elif (first_order == sell) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread):
        entry = mt5.symbol_info_tick(symbol).bid
        top = entry + (a+1) * x *point*10
        bottom = entry - a *x *point*10
        position = sell(symbol,lot=lot)
        positions.append(position)
        step = 1
        print('order 1 executed ')
        sleep(0.5)
        while first_order==sell:
            if ((mt5.positions_get()[-1]._asdict()['type'] == 1) and (mt5.symbol_info_tick(symbol).ask >= entry + x*point*10) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                lot = 2 * lot
                position = buy(symbol, lot)
                positions.append(position)
                step = step + 1
                print('order %i executed'%(step))
                sleep(0.5)
            elif ((mt5.positions_get()[-1]._asdict()['type'] == 0) and (mt5.symbol_info_tick(symbol).bid <= entry) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                lot = 2 * lot
                position = sell(symbol, lot)
                positions.append(position)
                sleep(0.5)
                step = step + 1
                print('order %i executed'%(step))
            if ((mt5.symbol_info_tick(symbol).bid < bottom) or (mt5.symbol_info_tick(symbol).ask >= top)):
                close_all_position(symbol ,positions)
                print('all %i orders closed'%(step))
                break
    return step

# run(first_order=buy, lot=0.1, a=2, x=3, symbol='EURUSD')

# setting max step

In [ ]:
def profit():
    Positions = mt5.positions_get(symbol='EURUSD')
    df=pd.DataFrame(list(Positions),columns=Positions[0]._asdict().keys())
    return df['profit'].sum()

In [ ]:
from time import sleep

def run(first_order, lot, a, x, symbol, spread, max_step):
    point = mt5.symbol_info(symbol).point
    positions = []

    if (first_order == buy) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread):
        entry = mt5.symbol_info_tick(symbol).ask
        top = entry + a * x *point*10
        bottom = entry - (a+1) * x *point*10
        position = buy(symbol,lot=lot)
        positions.append(position)
        step = 1
        print('order 1 executed ')
        sleep(0.5)
        while first_order==buy:
            if ((mt5.positions_get()[-1]._asdict()['type'] == 0) and (mt5.symbol_info_tick(symbol).bid <= entry - x*point*10) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread) and (step < max_step)):
                lot = 2 * lot
                position = sell(symbol, lot)
                positions.append(position)
                step = step + 1
                print('order %i executed'%(step))
                sleep(0.5)
            elif ((mt5.positions_get()[-1]._asdict()['type'] == 1) and (mt5.symbol_info_tick(symbol).ask >= entry) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread) and (step < max_step)):
                lot = 2 * lot
                position = buy(symbol, lot)
                positions.append(position)
                sleep(0.5)
                step = step + 1
                print('order %i executed'%(step))
            if ((mt5.symbol_info_tick(symbol).ask < bottom) or (mt5.symbol_info_tick(symbol).bid >= top)):
                close_all_position(symbol ,positions)
                print('all orders has closed at step %i'%(step))
                break
            while (step >= max_step) and (profit() < 0) and (entry < top):
                top = entry
                if ((mt5.symbol_info_tick(symbol).ask < bottom) or (mt5.symbol_info_tick(symbol).bid > top)):
                    print('top has shifted on %f ' %(top))
                    break
            while (step >= max_step) and (profit() >= 0) :
                top = entry - x*point*10
                if ((mt5.symbol_info_tick(symbol).ask < bottom) or (mt5.symbol_info_tick(symbol).bid > top)):
                    print('top has shifted on %f ' %(top))
                    break



    elif (first_order == sell) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread):
        entry = mt5.symbol_info_tick(symbol).bid
        top = entry + (a+1) * x *point*10
        bottom = entry - a *x *point*10
        position = sell(symbol,lot=lot)
        positions.append(position)
        step = 1
        print('order 1 executed ')
        sleep(0.5)
        while first_order==sell:
            if ((mt5.positions_get()[-1]._asdict()['type'] == 1) and (mt5.symbol_info_tick(symbol).ask >= entry + x*point*10) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread) and (step < max_step)):
                lot = 2 * lot
                position = buy(symbol, lot)
                positions.append(position)
                step = step + 1
                print('order %i executed'%(step))
                sleep(0.5)
            elif ((mt5.positions_get()[-1]._asdict()['type'] == 0) and (mt5.symbol_info_tick(symbol).bid <= entry) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread) and (step < max_step)):
                lot = 2 * lot
                position = sell(symbol, lot)
                positions.append(position)
                sleep(0.5)
                step = step + 1
                print('order %i executed'%(step))
            if ((mt5.symbol_info_tick(symbol).bid < bottom) or (mt5.symbol_info_tick(symbol).ask >= top)):
                close_all_position(symbol ,positions)
                print('all orders has closed at step %i'%(step))
                break
            while (step >= max_step) and (profit() < 0) and (entry > bottom):
                bottom = entry
                if ((mt5.symbol_info_tick(symbol).bid < bottom) or (mt5.symbol_info_tick(symbol).ask > top)):
                    print('bottom has shifted on %f ' %(bottom))
                    break
            while (step >= max_step) and (profit() >= 0) :
                bottom = entry + x*point*10
                if ((mt5.symbol_info_tick(symbol).bid < bottom) or (mt5.symbol_info_tick(symbol).ask > top)):
                    print('bottom has shifted on %f ' %(bottom))
                    break

    return step

# run(first_order=buy, lot=0.1, a=2, x=3, symbol='EURUSD')

# indic recom

In [7]:
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta

def get_recom(time_frame):
    if time_frame == '1m':
        pair = TA_Handler(symbol="EURUSD", screener="forex", exchange="FX_IDC", interval=Interval.INTERVAL_1_MINUTE )
    elif time_frame == '5m' :
        pair = TA_Handler(symbol="EURUSD", screener="forex", exchange="FX_IDC", interval=Interval.INTERVAL_5_MINUTES )
    elif time_frame == '15m' :
        pair = TA_Handler(symbol="EURUSD", screener="forex", exchange="FX_IDC", interval=Interval.INTERVAL_15_MINUTES )
    elif time_frame == '1h' :
        pair = TA_Handler(symbol="EURUSD", screener="forex", exchange="FX_IDC", interval=Interval.INTERVAL_1_HOUR )
    elif time_frame == '4h' :
        pair = TA_Handler(symbol="EURUSD", screener="forex", exchange="FX_IDC", interval=Interval.INTERVAL_4_HOURS )
    elif time_frame == '1D' :
        pair = TA_Handler(symbol="EURUSD", screener="forex", exchange="FX_IDC", interval=Interval.INTERVAL_1_DAY )

    return pair.get_analysis().summary['RECOMMENDATION']


In [8]:
from time import sleep
from datetime import datetime
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta

cycles = {}
while get_recom('1m') != 'NEUTRAL':
    for i in range(1,101):
        start = datetime.now()
        if get_recom('1m') == ('BUY' or 'STRONG_BUY'):
            recom = get_recom('1m')
            print('first order of cycle %i is %s' %(i,recom))
            step = run(buy, lot=0.1, a=2, x=5, symbol="EURUSD", spread=5)


        elif get_recom('1m') == ('SELL' or 'STRONG_SELL') :
            recom = get_recom('1m')
            print('first order of cycle %i is %s' %(i,recom))
            step = run(sell, lot=0.1, a=2, x=5, symbol='EURUSD', spread=5)

        end = datetime.now()

        cycles[i] = {'start' : start , 'end':end , 'step':step , 'recommendation':recom}
        print('cycles %i has ended --------------' %(i))
        i = i + 1

print('current recom is', get_recom('1m'))

first order of cycle 1 is BUY
order 1 executed 
order 2 executed
order 3 executed
all 3 orders closed
cycles 1 has ended --------------
cycles 2 has ended --------------
cycles 3 has ended --------------
cycles 4 has ended --------------
cycles 5 has ended --------------
cycles 6 has ended --------------
cycles 7 has ended --------------
cycles 8 has ended --------------


Exception: Can't access TradingView's API. HTTP status code: 403. Check for invalid symbol, exchange, or indicators.

In [ ]:
from time import sleep
from datetime import datetime
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta

cycles = {}

while get_recom('1m') != 'NEUTRAL':
    for i in range(1,101):
        start = datetime.now()
        if get_recom('1m') == ('BUY' or 'STRONG_BUY'):
            recom = get_recom('1m')
            print('first order of cycle %i is %s' %(i,recom))
            step = run(buy, lot=0.1, a=2, x=5, symbol="EURUSD", spread=5, max_step=6)


        elif get_recom('1m') == ('SELL' or 'STRONG_SELL') :
            recom = get_recom('1m')
            print('first order of cycle %i is %s' %(i,recom))
            step = run(sell, lot=0.1, a=2, x=5, symbol='EURUSD', spread=5, max_step=6)

        end = datetime.now()

        cycles[i] = {'start' : start , 'end':end , 'step':step , 'recommendation':recom}
        print('cycles %i has ended --------------' %(i))
        i = i + 1

# customize volume

In [ ]:
from time import sleep

def run(first_order, a, x, symbol, spread, volume):
    point = mt5.symbol_info(symbol).point
    positions = []

    if (first_order == buy) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread):
        entry = mt5.symbol_info_tick(symbol).ask
        top = entry + a * x *point*10
        bottom = entry - (a+1) * x *point*10
        position = buy(symbol,lot=volume[0])
        positions.append(position)
        step = 1
        print('first order executed ')
        sleep(1)
        while (first_order==buy) and (step < len(volume)):
            if ((mt5.positions_get()[-1]._asdict()['type'] == 0) and (mt5.symbol_info_tick(symbol).bid <= entry - x*point*10) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                position = sell(symbol, volume[step])
                positions.append(position)
                step = step + 1
                print('order %i executed'%(step))
                sleep(0.5)
            elif ((mt5.positions_get()[-1]._asdict()['type'] == 1) and (mt5.symbol_info_tick(symbol).ask >= entry) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                position = buy(symbol, volume[step])
                positions.append(position)
                sleep(0.5)
                step = step + 1
                print('order %i executed'%(step))
            if ((mt5.symbol_info_tick(symbol).ask < bottom) or (mt5.symbol_info_tick(symbol).bid >= top)):
                close_all_position(symbol ,positions)
                print('all %i orders closed'%(step))
                break



    elif (first_order == sell) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread):
        entry = mt5.symbol_info_tick(symbol).bid
        top = entry + (a+1) * x *point*10
        bottom = entry - a *x *point*10
        position = sell(symbol,lot=volume[0])
        positions.append(position)
        step = 1
        print('first order executed ')
        sleep(0.5)
        while first_order==sell and (step < len(volume)):
            if ((mt5.positions_get()[-1]._asdict()['type'] == 1) and (mt5.symbol_info_tick(symbol).ask >= entry + x*point*10) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                position = buy(symbol, volume[step])
                positions.append(position)
                step = step + 1
                print('order %i executed'%(step))
                sleep(0.5)
            elif ((mt5.positions_get()[-1]._asdict()['type'] == 0) and (mt5.symbol_info_tick(symbol).bid <= entry) and (mt5.symbol_info(symbol)._asdict()['spread'] <= spread)):
                position = sell(symbol, volume[step])
                positions.append(position)
                sleep(0.5)
                step = step + 1
                print('order %i executed'%(step))
            if ((mt5.symbol_info_tick(symbol).bid < bottom) or (mt5.symbol_info_tick(symbol).ask >= top)):
                close_all_position(symbol ,positions)
                print('all %i orders closed'%(step))
                break
    return step

# run(first_order=buy, lot=0.1, a=2, x=3, symbol='EURUSD')